# Turnout y votos a candidatos

La pregunta es:
¿Existe una relación entre el nivel de participación electoral (turnout) en los distritos electorales y la proporción de votos obtenidos por el candidato perdedor?

La pregunta está inspirada en este artículo: https://ladiaria.com.uy/elecciones/articulo/2024/6/el-enigma-de-las-internas-la-distancia-entre-orsi-y-cosse/

y podría ser un análisis que le sirva a distintos candidatos en el futuro: ¿Me conviene o no me conviene la alta participación?

Analicemos el caso de Carolina Cosse y el Frente Amplio.

In [1]:
import pandas as pd

In [3]:
#Sirve para montar el drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

MessageError: Error: credential propagation was unsuccessful

In [ ]:
directorio="/content/drive/MyDrive/EDA_course/Clase 1/data/raw/"

In [ ]:
# 2. Leer la Data
archivo = directorio + 'ODN Total Pais Circuito Hoja_0.xlsx'
# `skiprows` se usa para saltar las filas iniciales
df = pd.read_excel(archivo, skiprows=8)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/EDA_course/Clase 1/data/raw/ODN Total Pais Circuito Hoja_0.xlsx'

In [ ]:
# @title
# 3.a Número de columnas
num_columnas = df.shape[1]
print(f'Número de columnas: {num_columnas}')

In [ ]:
# @title
# 3.b Número de filas
num_filas = df.shape[0]
print(f'Número de filas: {num_filas}')

In [ ]:
# @title
# 4. Ejecutar .info() y .head()
df.info()
print(df.head())

In [ ]:
# @title
# 5. Ver la parte superior e inferior de los datos
print("Parte superior del dataset:")
print(df.head())
print("Parte inferior del dataset:")
print(df.tail())

In [ ]:
# @title
# 6. Verificar los n's. En Uruguay hay unos 7000 circuitos.
#Contar cuántas veces aparece cada PRECANDIDATO en las observaciones únicas del par (SERIE, CIRCUITO)
# Agrupamos por PRECANDIDATO y contamos las observaciones únicas del par (SERIE, CIRCUITO)

# Creamos una nueva columna que combina SERIE y CIRCUITO para identificar las tuples únicas
df['SERIE_CIRCUITO'] = df[['SERIES', 'CIRCUITO']].apply(tuple, axis=1)

# Eliminamos duplicados para contar las tuples únicas por PRECANDIDATO
df_unicos = df.drop_duplicates(subset=['PRECANDIDATO', 'SERIE_CIRCUITO'])

# Contamos cuántas veces aparece cada PRECANDIDATO en las tuples únicas de SERIE y CIRCUITO
conteo_precandidato = df_unicos['PRECANDIDATO'].value_counts()
print("Número de veces que cada PRECANDIDATO aparece en observaciones únicas de (SERIE, CIRCUITO):")
print(conteo_precandidato)

In [ ]:
# @title
# 7. Validar con datos externos. Sabemos cuánto sacó cada partido, y cuánto sacó cada candidato.
# Sumar todos los valores de CNT_VOTOS por CONVOCATORIA
# Sumar todos los valores de CNT_VOTOS por PRECANDIDATO
# Calcular los shares de cada PRECANDIDATO dentro de cada CONVOCATORIA

# Suma de CNT_VOTOS por CONVOCATORIA
suma_votos_por_convocatoria = df.groupby('CONVOCATORIA')['CNT_VOTOS'].sum()
print("Suma de CNT_VOTOS por CONVOCATORIA:")
print(suma_votos_por_convocatoria)

# Suma de CNT_VOTOS por PRECANDIDATO dentro de cada CONVOCATORIA
suma_votos_por_precandidato_y_convocatoria = df.groupby(['CONVOCATORIA', 'PRECANDIDATO'])['CNT_VOTOS'].sum()

# Calcular las participaciones de cada PRECANDIDATO dentro de cada CONVOCATORIA
participacion_por_precandidato = suma_votos_por_precandidato_y_convocatoria / suma_votos_por_convocatoria


# Shares de cada pre candidato
print(participacion_por_precandidato)

In [ ]:
suma_votos_por_convocatoria.sum()

In [ ]:
# @title
# 8. Primer intento de abordaje de nuestra pregunta. En los circuitos que hubo más votos al FA, sacó más votos Carolina Cosse?
# Plot de votos de Carolina Cosse vs. Votos al FA.

# Construir un nuevo dataset que sume CNT_VOTOS por PRECANDIDATO y CONVOCATORIA por cada CIRCUITO
df_sumado = df.groupby(['SERIE_CIRCUITO', 'PRECANDIDATO', 'CONVOCATORIA'])['CNT_VOTOS'].sum().reset_index()

# Filtrar para "COSSE GARRIDO, Ana Carolina" y "Frente_Amplio_-_O.D.N."
df_cosse = df_sumado[df_sumado['PRECANDIDATO'] == "COSSE GARRIDO, Ana Carolina"]
df_fa = df_sumado[df_sumado['CONVOCATORIA'] == "Frente_Amplio_-_O.D.N."].groupby('SERIE_CIRCUITO').sum().reset_index()

# Asegurarse de que cada SERIE_CIRCUITO en df_fa tenga un valor para Carolina Cosse.
# Si no tiene votos, le asignamos un valor de 0
df_cosse_filled = df_fa[['SERIE_CIRCUITO']].merge(df_cosse, on='SERIE_CIRCUITO', how='left')
df_cosse_filled['CNT_VOTOS'] = df_cosse_filled['CNT_VOTOS'].fillna(0)

# Unir los datos de Cosse y Frente Amplio
df_final = df_cosse_filled.merge(df_fa[['SERIE_CIRCUITO', 'CNT_VOTOS']], on='SERIE_CIRCUITO', suffixes=('_COSSE', '_FA'))


In [ ]:
# @title
# Realizar el gráfico
import matplotlib.pyplot as plt

In [ ]:
# @title
# Realizar el gráfico
plt.figure(figsize=(10, 6))
plt.scatter(df_final['CNT_VOTOS_FA'], df_final['CNT_VOTOS_COSSE'], alpha=0.1)
plt.xlabel('Suma de CNT_VOTOS para CONVOCATORIA == "Frente_Amplio_-_O.D.N."')
plt.ylabel('Suma de CNT_VOTOS para "COSSE GARRIDO, Ana Carolina"')
plt.title('Comparación de CNT_VOTOS por SERIE_CIRCUITO')
plt.show()

In [ ]:
# @title
import numpy as np

In [ ]:
# @title
# Extraer valores de x y y para la regresión
x = df_final['CNT_VOTOS_FA']
y = df_final['CNT_VOTOS_COSSE']

# Calcular la línea de regresión
slope, intercept = np.polyfit(x, y, 1)
regression_line = slope * x + intercept

# Realizar el gráfico
plt.figure(figsize=(10, 6))
plt.scatter(x, y, alpha=0.05, label='Datos')
plt.plot(x, regression_line, color='red', label='Línea de Regresión')
plt.xlabel('Suma de CNT_VOTOS para CONVOCATORIA == "Frente_Amplio_-_O.D.N."')
plt.ylabel('Suma de CNT_VOTOS para "COSSE GARRIDO, Ana Carolina"')
plt.title('Comparación de CNT_VOTOS por SERIE_CIRCUITO')
plt.legend()
plt.show()

In [ ]:
# @title
